In [46]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pandas as pd
import time

In [47]:
df_population = pd.read_csv("world-city-listing-table.csv")

In [48]:
# Initialize geocoder
geolocator = Nominatim(user_agent="job_location_parser")

def _parse_location(location):
    # Handle NA or None values
    if pd.isna(location) or location.upper() == "NA":
        return ("Unknown", "Unknown")

    # Handle remote jobs separately
    if "REMOTE" in location.upper():
        return ("Remote", "Remote")
    
    # Handle remote jobs with no location specified separately
    if location.upper() == "NONE":
        return ("Unknown", "Unknown")

    attempt = 0
    while attempt < 5:
        try:
            # Geocode the location
            location_geo = geolocator.geocode(location, language='en')
            if location_geo:
                # Split the address into components
                address_parts = location_geo.address.split(',')
                # Extract the relevant parts
                country = address_parts[-1].strip()
                city = address_parts[0].strip() if len(address_parts) > 0 else "Unknown"
                
                # Check for administrative areas in the address parts
                if len(address_parts) > 2:
                    if any(keyword in address_parts[-3].lower() for keyword in ["city", "town", "village", "municipality"]):
                        city = address_parts[-3].strip()
                    elif any(keyword in address_parts[-2].lower() for keyword in ["city", "town", "village", "municipality"]):
                        city = address_parts[-2].strip()
                return (city, country)
            else:
                return ("Unknown", "Unknown")
        except GeocoderTimedOut:
            attempt += 1
            print(f"Timeout occurred for {location}. Retrying... (Attempt {attempt})")
            time.sleep(2 ** attempt)  # Exponential backoff
        except Exception as e:
            print(f"Exception: {e}")
            return ("Unknown", "Unknown")
    return ("Unknown", "Unknown")

In [49]:
# Set all columns to lowercase 
df_population.columns = map(str.lower, df_population.columns)
# combine city and country columns into location column
df_population['location'] = df_population['city'] + ", " + df_population['country']

# Handle location column
df_population[['city_geopy', 'country_geopy']] = df_population['location'].apply(lambda x: pd.Series(_parse_location(x)))

# Set values in all columns to lowercase
df_population = df_population.apply(lambda x: x.astype(str).str.lower())

# save
df_population.to_csv("world-city-listing-table-geopy-raw.csv", index=False)

In [50]:
# print all the unique locations that could not be geocoded
print(df_population[df_population['city_geopy'] == "unknown"]['location'].unique())

  population   pop2024   pop2023       city     country growthrate type rank  \
0   37115035  37115035  37194105      tokyo       japan    -0.0021    w    1   
1   33807403  33807403  32941309      delhi       india     0.0263    w    2   
2   29867918  29867918  29210808   shanghai       china     0.0225    w    3   
3   23935652  23935652  23209616      dhaka  bangladesh     0.0313    w    4   
4   22806704  22806704  22619736  sao paulo      brazil     0.0083    w    5   

            location city_geopy country_geopy  
0       tokyo, japan      tokyo         japan  
1       delhi, india      delhi         india  
2    shanghai, china   shanghai         china  
3  dhaka, bangladesh      dhaka    bangladesh  
4  sao paulo, brazil  são paulo        brazil  


In [68]:
# clean df_population remove rows with unknown city_geopy
df_population = df_population[df_population['city_geopy'] != "unknown"]
# save cleaned df_population
df_population.to_csv("world-city-listing-table-geopy-clean.csv", index=False)

In [69]:
fw_data_raw = pd.read_csv("findwork_data_raw.csv")

In [70]:
# print all unique values in the location column
# print(fw_data_raw['location'].unique())

In [71]:
# filter keep only remote = false
fw_data = fw_data_raw[fw_data_raw['remote'] == False]
# remove location = nan
fw_data = fw_data[fw_data['location'].notna()]

# if locations contains remote, set remote to true
fw_data['remote'] = fw_data['location'].apply(lambda x: True if "remote" in x.lower() else False)
# filter keep only remote = false
fw_data = fw_data[fw_data['remote'] == False]

In [83]:
fw_data.drop_duplicates(subset=['location'])

,id,role,company_name,company_num_employees,employment_type,location,remote,logo,url,text,date_posted,keywords,source,city,country
1,M1pNwaQ,mid+ engineering roles,GA-CCRi,NaN,NaN,"onsite (hybrid or full): VA, DC, UT",False,NaN,https://findwork.dev/M1pNwaQ/mid-engineering-r...,We usually hire remote (USA) but we&#x27;ve pa...,2024-03-01T16:49:29Z,"{k8s,rust}",Hn,Unknown,Unknown
19,M3GNp2M,"Senior/Staff Engineer, Lead Frontend Engineer,...",Hashboard,NaN,NaN,New York (NYC),False,NaN,https://findwork.dev/M3GNp2M/seniorstaff-engin...,Hashboard is building a BI platform that makes...,2024-03-01T19:36:33Z,{react},Hn,New York,United States
30,M3GYPBM,Biomedical Engineer // Quantune Technologies GmbH,Quantune Technologies,NaN,full time,"Berlin, Germany",False,https://findwork-dev-images.s3.amazonaws.com/f...,https://findwork.dev/M3GYPBM/biomedical-engine...,<em>About us:</em>\r\n\r\nQuantune Technologie...,2024-03-27T04:00:00Z,"{python,embedded}",Berlinstartupjobs,Berlin,Germany
48,MAdNAdQ,Full-stack / Frontend Engineer,Napper,NaN,NaN,Stockholm,False,NaN,https://findwork.dev/MAdNAdQ/full-stack-fronte...,"We are a small, highly ambitious and fun (argu...",2024-03-01T16:24:52Z,"{react,ml,typescript,serverless,python}",Hn,Stockholm,Sweden
49,MAdNwVQ,"Machine Learning Engineer, Data Engineer",ML6,NaN,NaN,"Amsterdam, Berlin, Ghent (EU) On-site/hybrid",False,https://findwork-dev-images.s3.amazonaws.com/ML6,https://findwork.dev/MAdNwVQ/machine-learning-...,We are a Machine Learning consulting company t...,2024-03-02T17:28:52Z,"{ml,gcp,pytorch,tensorflow,aws,python,azure}",Hn,Unknown,Unknown
68,MBjrE1M,Risk Manager - Liquid Token Fund,ARP Digital,NaN,full time,"Dubai, United Arab Emirates",False,https://findwork-dev-images.s3.amazonaws.com/f...,https://findwork.dev/MBjrE1M/risk-manager-liqu...,<p>We are seeking an experienced individual wi...,2024-04-06T04:00:00Z,{blockchain},Cryptocurrencyjobs,Dubai,United Arab Emirates
74,MBjRzBM,Platform Engineers,prokube.ai,NaN,NaN,Northern Germany (Hamburg or Bremen),False,NaN,https://findwork.dev/MBjRzBM/platform-engineer...,We are building an integrated MLOps platform b...,2024-05-01T21:01:26Z,"{k8s,kubernetes}",Hn,Unknown,Unknown
86,MdV9PDn,Senior Industrial Designer,Solventum,NaN,NaN,"Maplewood, MN",False,NaN,https://findwork.dev/MdV9PDn/senior-industrial...,<h3>Job description</h3>3M (New Health Care Co...,2024-04-15T20:08:00Z,{embedded},Coroflot,Maplewood,United States
88,MdV9xWn,Multiple Roles,Seen Finance,NaN,NaN,Berlin,False,NaN,https://findwork.dev/MdV9xWn/multiple-roles-at...,"At Seen, we believe that everyone deserves a f...",2024-04-02T11:23:18Z,"{react,fintech,typescript,python}",Hn,Berlin,Germany
91,MdVg2xn,Senior Developer Experience Engineer,Novata,NaN,NaN,"London, UK (Hybrid)",False,NaN,https://findwork.dev/MdVg2xn/senior-developer-...,Novata is pioneering the way in ESG (Environme...,2024-03-01T18:23:28Z,"{k8s,typescript}",Hn,Unknown,Unknown


In [82]:
# Handle location column - create a table to map the original location to the city and country
# get only unique locations

map_location = fw_data.drop_duplicates(subset=['location'])
map_location[['city_geopy', 'country_geopy']] = map_location['location'].apply(lambda x: pd.Series(_parse_location(x)))

# Set values in all columns to lowercase
map_location = map_location.apply(lambda x: x.astype(str).str.lower())

C:\Users\jessicas\AppData\Local\Temp\ipykernel_21192\2342724613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_location[['city_geopy', 'city_geopy']] = map_location['location'].apply(lambda x: pd.Series(_parse_location(x)))


In [80]:
map_location = map_location[['location', 'city_geopy', 'country_geopy']]

In [81]:
# convert location to lowercase
fw_data_raw['location'] = fw_data_raw['location'].str.lower()

test = pd.merge(fw_data_raw, map_location, on='location', how='left')

In [37]:
# remove all unknown locations
all_locations_clean = map_location[map_location['city'] != 'unknown']

# read_file = pd.read_csv("pop2.csv")
# # group by city and country and keep the record with the highest population in the test df
# idx = read_file.groupby(['city2', 'country2'])['rank'].idxmax()
# pop_summarise = read_file.loc[idx].reset_index(drop=True)
# # remove all unknown locations
# pop_summarise_clean = pop_summarise[pop_summarise['city2'] != 'unknown']


In [40]:
merged = pd.merge(all_locations_clean, pop_summarise_clean, how='left', left_on=['city', 'country'], right_on=['city2', 'country2'])

# select only the columns we need
merged = merged[['location','city_y', 'country_y','city2', 'country2']]

In [28]:
# group by city and country and count in the test df
pop_summarise = df_jobs_renamed.groupby(['city2', 'country2']).size().reset_index(name='count')

# group by city and country and keep the record with the highest rank in the fw_data df
# pop_summarise = test.groupby(['city', 'country']).apply(lambda x: x[x['rank'].idxmax()])

# group by city and country and keep the record with the highest population in the test df
# idx = test.groupby(['city2', 'country2'])['population'].idxmax()
# pop_summarise = test.loc[idx].reset_index(drop=True)
